In [1]:
import pandas as pd
import re #regex library
import nltk
import string
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [2]:
TWEET_DATA = pd.read_excel('dataSet.xlsx')
TWEET_DATA

,created_at,username,text
0,2020-06-16,collegemenfess,"[cm] guys mau nanya, temenku kan ngirim surat ..."
1,2020-06-16,flasvh,@collegemenfess Mngkin karena daring makanya s...
2,2020-06-16,AMWalisongo,"Dengan adanya survei kecil-kecilan ini, lebih ..."
3,2020-06-16,rogasper,"@vitobasterbine Saya juga setuju, Kasihan yg s..."
4,2020-06-16,delacr0ixx,"RT @bahrul04_: Setelah kuliah daring, yang dip..."
...,...,...,...
582,2020-06-21,AriessandyIrvan,@official_unila Jadi unila kuliah daring apa g...
583,2020-06-21,ipbofficial,"Selain itu, Rektor juga menjelaskan bahwa kebi..."
584,2020-06-21,lasa666na,kuliah daring di rumah semakin berat karena ke...
585,2020-06-21,FerdianRifaldi,Ada 2 orang yang mau mengenalkan temannya ke a...


In [3]:
##case folding
TWEET_DATA['text'] = TWEET_DATA['text'].str.lower()
TWEET_DATA

,created_at,username,text
0,2020-06-16,collegemenfess,"[cm] guys mau nanya, temenku kan ngirim surat ..."
1,2020-06-16,flasvh,@collegemenfess mngkin karena daring makanya s...
2,2020-06-16,AMWalisongo,"dengan adanya survei kecil-kecilan ini, lebih ..."
3,2020-06-16,rogasper,"@vitobasterbine saya juga setuju, kasihan yg s..."
4,2020-06-16,delacr0ixx,"rt @bahrul04_: setelah kuliah daring, yang dip..."
...,...,...,...
582,2020-06-21,AriessandyIrvan,@official_unila jadi unila kuliah daring apa g...
583,2020-06-21,ipbofficial,"selain itu, rektor juga menjelaskan bahwa kebi..."
584,2020-06-21,lasa666na,kuliah daring di rumah semakin berat karena ke...
585,2020-06-21,FerdianRifaldi,ada 2 orang yang mau mengenalkan temannya ke a...


In [4]:
##pengahpusan simbol
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_tweet_special)
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['text'] = TWEET_DATA['text'].apply(remove_singl_char)
TWEET_DATA['created_at'] = TWEET_DATA['created_at'].apply(lambda a: pd.to_datetime(a).date())
TWEET_DATA

,created_at,username,text
0,2020-06-16,collegemenfess,cm guys mau nanya temenku kan ngirim surat eda...
1,2020-06-16,flasvh,mngkin karena daring makanya sempurna semua ni...
2,2020-06-16,AMWalisongo,dengan adanya survei kecilkecilan ini lebih ba...
3,2020-06-16,rogasper,saya juga setuju kasihan yg sulit punya akses ...
4,2020-06-16,delacr0ixx,rt setelah kuliah daring yang dipikiran gua ad...
...,...,...,...
582,2020-06-21,AriessandyIrvan,unila jadi unila kuliah daring apa gimana min
583,2020-06-21,ipbofficial,selain itu rektor juga menjelaskan bahwa kebij...
584,2020-06-21,lasa666na,kuliah daring di rumah semakin berat karena ke...
585,2020-06-21,FerdianRifaldi,ada orang yang mau mengenalkan temannya ke aku...


In [5]:
df_preprocessed = TWEET_DATA.copy()
df_preprocessed = df_preprocessed.drop(columns=['created_at'])
df_preprocessed.head()

,username,text
0,collegemenfess,cm guys mau nanya temenku kan ngirim surat eda...
1,flasvh,mngkin karena daring makanya sempurna semua ni...
2,AMWalisongo,dengan adanya survei kecilkecilan ini lebih ba...
3,rogasper,saya juga setuju kasihan yg sulit punya akses ...
4,delacr0ixx,rt setelah kuliah daring yang dipikiran gua ad...


In [6]:
df_preprocessed

,username,text
0,collegemenfess,cm guys mau nanya temenku kan ngirim surat eda...
1,flasvh,mngkin karena daring makanya sempurna semua ni...
2,AMWalisongo,dengan adanya survei kecilkecilan ini lebih ba...
3,rogasper,saya juga setuju kasihan yg sulit punya akses ...
4,delacr0ixx,rt setelah kuliah daring yang dipikiran gua ad...
...,...,...
582,AriessandyIrvan,unila jadi unila kuliah daring apa gimana min
583,ipbofficial,selain itu rektor juga menjelaskan bahwa kebij...
584,lasa666na,kuliah daring di rumah semakin berat karena ke...
585,FerdianRifaldi,ada orang yang mau mengenalkan temannya ke aku...


In [8]:
#tokenisasi 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df_preprocessed['tweet_tokens'] = df_preprocessed['text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df_preprocessed['tweet_tokens'].head())
print('\n\n\n')
# TWEET_DATA

Tokenizing Result : 

0    [cm, guys, mau, nanya, temenku, kan, ngirim, s...
1    [mngkin, karena, daring, makanya, sempurna, se...
2    [dengan, adanya, survei, kecilkecilan, ini, le...
3    [saya, juga, setuju, kasihan, yg, sulit, punya...
4    [rt, setelah, kuliah, daring, yang, dipikiran,...
Name: tweet_tokens, dtype: object






In [9]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

df_preprocessed['tweet_tokens_fdist'] = df_preprocessed['tweet_tokens'].apply(freqDist_wrapper)
print('Frequency Tokens : \n') 
print(df_preprocessed['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))
# TWEET_DATA

Frequency Tokens : 

0    [(cm, 1), (guys, 1), (mau, 1), (nanya, 1), (te...
1    [(sempurna, 2), (mngkin, 1), (karena, 1), (dar...
2    [(dengan, 1), (adanya, 1), (survei, 1), (kecil...
3    [(saya, 1), (juga, 1), (setuju, 1), (kasihan, ...
4    [(rt, 1), (setelah, 1), (kuliah, 1), (daring, ...
Name: tweet_tokens_fdist, dtype: object


In [11]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df_preprocessed['tweet_tokens_WSW'] = df_preprocessed['tweet_tokens'].apply(stopwords_removal) 


print(df_preprocessed['tweet_tokens_WSW'].head())

0    [nanya, temenku, ngirim, surat, edaran, kemend...
1    [mngkin, daring, sempurna, nilai, gua, heheheh...
2    [survei, kecilkecilan, milih, kuliah, tatap, m...
3    [setuju, kasihan, sulit, akses, internet, kuli...
4      [kuliah, daring, dipikiran, gua, minus, nambah]
Name: tweet_tokens_WSW, dtype: object


In [12]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(dokumen):
    return [StemmerFactory().create_stemmer().stem(kata) for kata in dokumen if kata and kata.isascii()]
df_preprocessed['steeming'] = df_preprocessed['tweet_tokens_WSW'].apply(stemming)
print(df_preprocessed['steeming'].head())

0    [nanya, temenku, ngirim, surat, edar, kemendik...
1    [mngkin, daring, sempurna, nilai, gua, heheheh...
2    [survei, kecilkecilan, milih, kuliah, tatap, m...
3    [tuju, kasihan, sulit, akses, internet, kuliah...
4          [kuliah, daring, pikir, gua, minus, nambah]
Name: steeming, dtype: object


In [13]:
df_preprocessed

,username,text,tweet_tokens,tweet_tokens_fdist,tweet_tokens_WSW,steeming
0,collegemenfess,cm guys mau nanya temenku kan ngirim surat eda...,"[cm, guys, mau, nanya, temenku, kan, ngirim, s...","{'cm': 1, 'guys': 1, 'mau': 1, 'nanya': 1, 'te...","[nanya, temenku, ngirim, surat, edaran, kemend...","[nanya, temenku, ngirim, surat, edar, kemendik..."
1,flasvh,mngkin karena daring makanya sempurna semua ni...,"[mngkin, karena, daring, makanya, sempurna, se...","{'mngkin': 1, 'karena': 1, 'daring': 1, 'makan...","[mngkin, daring, sempurna, nilai, gua, heheheh...","[mngkin, daring, sempurna, nilai, gua, heheheh..."
2,AMWalisongo,dengan adanya survei kecilkecilan ini lebih ba...,"[dengan, adanya, survei, kecilkecilan, ini, le...","{'dengan': 1, 'adanya': 1, 'survei': 1, 'kecil...","[survei, kecilkecilan, milih, kuliah, tatap, m...","[survei, kecilkecilan, milih, kuliah, tatap, m..."
3,rogasper,saya juga setuju kasihan yg sulit punya akses ...,"[saya, juga, setuju, kasihan, yg, sulit, punya...","{'saya': 1, 'juga': 1, 'setuju': 1, 'kasihan':...","[setuju, kasihan, sulit, akses, internet, kuli...","[tuju, kasihan, sulit, akses, internet, kuliah..."
4,delacr0ixx,rt setelah kuliah daring yang dipikiran gua ad...,"[rt, setelah, kuliah, daring, yang, dipikiran,...","{'rt': 1, 'setelah': 1, 'kuliah': 1, 'daring':...","[kuliah, daring, dipikiran, gua, minus, nambah]","[kuliah, daring, pikir, gua, minus, nambah]"
...,...,...,...,...,...,...
582,AriessandyIrvan,unila jadi unila kuliah daring apa gimana min,"[unila, jadi, unila, kuliah, daring, apa, gima...","{'unila': 2, 'jadi': 1, 'kuliah': 1, 'daring':...","[unila, unila, kuliah, daring]","[unila, unila, kuliah, daring]"
583,ipbofficial,selain itu rektor juga menjelaskan bahwa kebij...,"[selain, itu, rektor, juga, menjelaskan, bahwa...","{'selain': 1, 'itu': 1, 'rektor': 1, 'juga': 1...","[rektor, kebijakan, perkuliahan, dilaksanakan,...","[rektor, bijak, kuliah, laksana, daring]"
584,lasa666na,kuliah daring di rumah semakin berat karena ke...,"[kuliah, daring, di, rumah, semakin, berat, ka...","{'kuliah': 1, 'daring': 1, 'di': 1, 'rumah': 1...","[kuliah, daring, rumah, berat, keluarga, dukun...","[kuliah, daring, rumah, berat, keluarga, dukun..."
585,FerdianRifaldi,ada orang yang mau mengenalkan temannya ke aku...,"[ada, orang, yang, mau, mengenalkan, temannya,...","{'ada': 1, 'orang': 1, 'yang': 2, 'mau': 1, 'm...","[mengenalkan, temannya, insecure, badan]","[kenal, teman, insecure, badan]"


In [14]:
from openpyxl import load_workbook
book = load_workbook('D:\Kuliah\Skripsi\Penelitian\dataSet.xlsx')
writer = pd.ExcelWriter('D:\Kuliah\Skripsi\Penelitian\dataSet.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

df_preprocessed.to_excel(writer, index = False)

writer.save()